# Uma RNN mínima a nível de caractere

Vinicius F. Caridá

Neste notebook nós treinamos uma RNN em algumas músicas da Anitta. Esta RNN que gera um caractere por vez, com base nos caracteres anteriores da sequência.

O código original da RNN foi escrit0 por Andrej Karpathy (atualmente diretor de IA da Tesla Motors).
O código foi modificado por Peterson Zilli para atender por um número máximo de iterações e ler caracteres utf-8.

Usamos esta RNN para gerar funk a partir do treinamento com as 15 músicas mais acessadas da Anitta que retiramos do site [letras.com.br](www.letras.com.br).

As letras estão no arquivo 'anitta.txt'

In [1]:
with open('anitta_15.txt', 'r', encoding='utf-8') as txt:
    print(txt.read(500))

Você prepara, mas não dispara
Você repara, mas não encara
Se acha o cara, mas não me para
Tá cheio de maldade, mas não me encara

Você já tá querendo e eu também
Mas é cheio de história e de porém
Virou covarde, tô com vontade
Mas você tá demorando uma eternidade

Se você não vem, eu vou botar pressão
Não vou te esperar, tô cheia de opção
Eu não sou mulher de aturar sermão
Me encara, se prepara
Que eu vou jogar bem na sua cara

Bem na sua cara
Eu vou rebolar bem na sua cara
Bem na sua cara
Hoje 


Agora, usamos este texto para treinar a rede recorrente

In [7]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License

Adapted for python 3.5 by Peterson Katagiri Zilli @petersonzilli
"""

num_iters_max = 100000


import numpy as np

# data I/O
data = open('anitta_15.txt', 'r', encoding='utf-8').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(('data has %d characters, %d unique.' % (data_size, vocab_size)).encode("utf-8"))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 256 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 0.01

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

#while True:
while n < num_iters_max:    
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('------------------------------------------------------------\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print ('iter %d, loss: %f\n\n' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

b'data has 17183 characters, 70 unique.'
------------------------------------------------------------
 p(-oRwÀLnxzóMOglàBo ó!.SzFtM!kNaFçéOãk"ÉÀá,wlhn,dNcôcFFÉxk,OtiT
JsmUErvÔHcÀhêp!eqrÀgdôBk
eófSv:ntQôCH")s.!xnrEhzILfTçcoIe.-RiQ,aqôak(p(qe nvrtqlÔicÔPxatljãnmêSwbeô"xdIMC!rómiháBçg!Aj?dçd.çAc zôA gsEmR 
----
iter 0, loss: 106.212381


------------------------------------------------------------
  o cenêro,
Vãican,
E t cêlgeitsao senar
Do oras nóendusnaa,
ADdneo de, bemalorsadejash J ter roqtedonmoito
Bu camidarsstrindeçlso
,u cormsvodo
Po poi ar pclsor erohané jaip ede oohaceroPa, O muça ho s 
----
iter 1000, loss: 80.440126


------------------------------------------------------------
 bel, dam tate são que aih
Nhos
Ion veu mhsra
Ae tagora peidcaim feCcr,para:sÉu bão ca zdo queze lacçrn
Beisso euando mem ua viminte mifomer
Pue vomam

Seibore
Der rejo quesbora xár boio
Eeloco me  ult 
----
iter 2000, loss: 64.740350


------------------------------------------------------------
 psaa 

------------------------------------------------------------
 bou, ra daise

E fão tô que da melia e beiper naro a mum
Der a vovil, nlovivar eu ssquile nasso cemeção
En a senlerad:, a a lia

E falo sei é me fazer o bol,
SBem talá que davir ala a bem
Par ar mei a 
----
iter 28000, loss: 32.242330


------------------------------------------------------------
 ela quer faiso a galutó
Ele batga pra tea

Elho te você
, rou
Ess o amis, elas bugobad
Eu fima su você

Boi só fai su mela camiçar, ele lua cemjosade fuco degar, mom uvtin


Cov, pela má ra doper

Ee  
----
iter 29000, loss: 32.050003


------------------------------------------------------------
  mindo e fácai faz iã) nto
Ais o ate velção
Eu gonta, tia, luecasenfado, bei
Mão beladir
Quando nhor?
Vei per
Uas ue jeganan sabai
Per em fico :in
Condem: comto nas vabae fucom o que ponde mó cêndofaz 
----
iter 30000, loss: 31.451168


------------------------------------------------------------
 m no suatzado que eu sio ela comepazbàr o

------------------------------------------------------------
 louqueçar
Ne bodá em ssor
Duo asinte de pra, passs:
Não se allendr mos que so amondo, rocê, eu veu teme arasso, coracordilo
Mem nem saque me pansanto
Eu de vim assprrdade eu tô que vam vim filo quer,  
----
iter 56000, loss: 22.511050


------------------------------------------------------------
 pado attevim assido co vomaror
O aques baijo Com vou pre elen emfina :Énto
ASzer eu aô bre amlolar
Me centosso deu
Pãa elemer que eu quero
Me do nté tar
Bem na suampradazaQ
ende pravola

Eu feçe jo mi 
----
iter 57000, loss: 21.620568


------------------------------------------------------------
 de m tar de praçar
Eu sebecar bas vachras
Vebêco
Meli que do pir
Dostro comer dis minher

Eu amor

Em no qua do coreçto
Nossade, canção
Quarona sá piter:
Josso jeste, losta arlia, eu quito
Esse lada m 
----
iter 58000, loss: 22.503211


------------------------------------------------------------
 a ele shora
De seu cimano deser

Tissa do

------------------------------------------------------------
  so seu lidar:
Bó dit nluda, fago eu pmozerda me parara dei tão lá eutaro
Messa  osso dezer que praseque enporquera
Vois ero dogonha r ia eu to ocalia
Eu ando tua care
Mem veixqmelocê pesfimo que é te 
----
iter 84000, loss: 18.061368


------------------------------------------------------------
 çar
Não se prrepas boc, sfor
Àão quero canto
A friço suo perde atlina é polha me dase dera adorpraquenda mensem
Tem jo eu cimevor rão pira é prontado
Bem desma compando e aviratr
Som iê pramai

Eó de  
----
iter 85000, loss: 17.255976


------------------------------------------------------------
 eu teu quero tesim bem, se bem não tide em gavaj iné meucar
Quemou não feger canin
Eu gontenvocaranto peu vis quela te re proder davass, eu tão quer na fuema em pissoi que vica meis é vor begorto or n 
----
iter 86000, loss: 17.832302


------------------------------------------------------------
 ito com o querta eu sai
Não fou tadinão m